In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
path = "C:/Users/Pedro Argento/Documents/Mestrado - Dissertação/data/"

In [8]:
pd.read_excel(path + '2019-03-01_15-12-06_Evolucao_historica_da__Media__na_serie_M__1_na_fonte_Convencional_SE_.xlsx', index_col=0)  # doctest: +SKIP

IOError: [Errno 2] No such file or directory: 'C:/Users/Pedro Argento/Documents/Mestrado - Disserta\xc3\xa7\xc3\xa3o/data/2019-03-01_15-12-06_Evolucao_historica_da__Media__na_serie_M__1_na_fonte_Convencional_SE_.xlsx'